In [20]:
from langchain.document_loaders import PyPDFLoader

#loads the pdf files from local storage
loaders = [
    PyPDFLoader("C:/db/rest.pdf"), 
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [26]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

chunk_size=150
chunk_overlap = 50

#splits the document into small parts with a size of 150 words
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
    )

splits = r_splitter.split_documents(docs)

In [29]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = 'BAAI/bge-small-en'
model_kwargs = {'device':'cpu'}
encode_kwargs = {'normalize_embeddings':True}

embedding = HuggingFaceBgeEmbeddings(
    model_name = model_name,
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs
    )

In [34]:
from langchain.vectorstores import Chroma

persist_directory = 'docs/chroma'

vectordb = Chroma.from_documents(
    documents = splits,
    embedding = embedding,
    persist_directory = persist_directory
    )

docs = vectordb.similarity_search("Where is it loacted",k=3)

In [40]:
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id = 'google/flan-t5-xxl',
    model_kwargs = {'temperature':0.5,'max_length':500},
    huggingfacehub_api_token = ""
    )


In [42]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = vectordb.as_retriever()
    )

In [43]:
result = qa_chain({'query':"Where is it loacted"})

C:\Users\VANSH KHANEJA\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [44]:
result['result']

'delhi, punjab, haryana, kerala,haryana'